<a href="https://colab.research.google.com/github/eghib22/Store-Sales-Forecasting/blob/main/model_experiment_prophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir ~/.kaggle
from google.colab import files
files.upload()
!mv "kaggle.json" ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json
!ls -l ~/.kaggle/

!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip walmart-recruiting-store-sales-forecasting
!unzip '*.csv.zip'
!unzip '*.csv.zip'

!pip install prophet
!pip install -q dagshub mlflow scikit-learn joblib


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/root/.kaggle’: File exists


Saving kaggle.json to kaggle.json
total 4
-rw------- 1 root root 71 Jul  7 13:19 kaggle.json
walmart-recruiting-store-sales-forecasting.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  walmart-recruiting-store-sales-forecasting.zip
replace features.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: features.csv.zip        
replace sampleSubmission.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sampleSubmission.csv.zip  
replace stores.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: stores.csv              
replace test.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv.zip            
replace train.csv.zip? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv.zip           
Archive:  test.csv.zip
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                

Archive:  features.csv.zip
replace features.csv? [y]es, [n]o, [A]ll, [N]one, [r]ena

In [ ]:
import dagshub
dagshub.init(repo_owner='eghib22', repo_name='Store-Sales-Forecasting', mlflow=True)

import mlflow
import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.getLogger("prophet").setLevel(logging.WARNING)

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import gc

from prophet import Prophet
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import joblib

train = pd.read_csv('train.csv')
features = pd.read_csv('features.csv')
stores = pd.read_csv('stores.csv')

train['Date'] = pd.to_datetime(train['Date'])
features['Date'] = pd.to_datetime(features['Date'])

df = train.merge(features, on=['Store', 'Date', 'IsHoliday'], how='left')
df = df.merge(stores, on='Store', how='left')
df = df.sort_values('Date')

for col in ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']:
    df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

def weighted_mae(y_true, y_pred, weights):
    return np.sum(weights * np.abs(y_true - y_pred)) / np.sum(weights)


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

Output()



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=b0ef0bca-80c7-46cd-af16-33e978c0e763&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=2214b5d2f9d41a81d7568d0a1e639ca20134ff31e573bcc95e98c7e25d9d624f




Accessing as agasi22

Initialized MLflow to track repo "eghib22/Store-Sales-Forecasting"

Repository eghib22/Store-Sales-Forecasting initialized!

In [ ]:
class ProphetWrapper(BaseEstimator, RegressorMixin):
    def __init__(self, yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False, changepoint_prior_scale=0.5):
        self.yearly_seasonality = yearly_seasonality
        self.weekly_seasonality = weekly_seasonality
        self.daily_seasonality = daily_seasonality
        self.changepoint_prior_scale = changepoint_prior_scale
        self.model_ = None

    def fit(self, X, y=None):
        self.model_ = Prophet(
            yearly_seasonality=self.yearly_seasonality,
            weekly_seasonality=self.weekly_seasonality,
            daily_seasonality=self.daily_seasonality,
            changepoint_prior_scale=self.changepoint_prior_scale
        )
        self.model_.fit(X)
        return self

    def predict(self, X):
        forecast = self.model_.predict(X)
        return forecast['yhat'].values


In [ ]:
results = []
all_preds = []

store_dept_groups = df.groupby(['Store', 'Dept'])
total_groups = len(store_dept_groups)

print(f"--- Starting Prophet for {total_groups} Store-Dept combos ---")

mlflow.set_experiment("Prophet_Forecasting")

with mlflow.start_run(run_name="Prophet_Improved_Seasonality_Run"):
    mlflow.log_param("model", "Prophet")
    mlflow.log_param("seasonality_yearly", True)
    mlflow.log_param("seasonality_weekly", True)
    mlflow.log_param("seasonality_daily", False)
    mlflow.log_param("changepoint_prior_scale", 0.5)

    for idx, ((store_id, dept_id), group) in enumerate(store_dept_groups, start=1):
        print(f"\n--- Processing Store: {store_id}, Dept: {dept_id} ({idx}/{total_groups}) ---")

        g = group.sort_values('Date').copy()
        g['ds'] = g['Date']
        g['y'] = g['Weekly_Sales']
        weights = g['IsHoliday'].apply(lambda x: 5 if x else 1)

        y_train = g[g['ds'] < '2012-01-01']
        y_val = g[(g['ds'] >= '2012-01-01') & (g['ds'] < '2012-07-01')]
        weights_val = weights.loc[y_val.index]

        if len(y_train) < 100 or len(y_val) < 20:
            print(f"   Skipped: Not enough data ({len(y_train)} train, {len(y_val)} val)")
            continue

        try:
            pipe = Pipeline([
                ('model', ProphetWrapper(
                    yearly_seasonality=True,
                    weekly_seasonality=True,
                    daily_seasonality=False,
                    changepoint_prior_scale=0.5
                ))
            ])

            pipe.fit(y_train[['ds', 'y']])
            y_pred = pipe.predict(y_val[['ds']])

            y_true = y_val['y'].values
            wmae = weighted_mae(y_true, y_pred, weights_val)
            rmse = np.sqrt(mean_squared_error(y_true, y_pred))

            print(f"   WMAE: {wmae:.2f} | RMSE: {rmse:.2f}")

            results.append({
                'Store': store_id,
                'Dept': dept_id,
                'RMSE': rmse,
                'WMAE': wmae
            })

            all_preds.append(pd.DataFrame({
                'y_true': y_true,
                'y_pred': y_pred,
                'weight': weights_val.values
            }))

        except Exception as e:
            print(f"   Failed: {e}")
            continue

        gc.collect()

    if len(all_preds) > 0:
        all_df = pd.concat(all_preds)
        overall_wmae = np.sum(all_df['weight'] * np.abs(all_df['y_true'] - all_df['y_pred'])) / np.sum(all_df['weight'])
        print(f"\n Overall WMAE: {overall_wmae:.2f}")
        mlflow.log_metric("Overall_WMAE", overall_wmae)
    else:
        print("No valid predictions generated.")

    mlflow.log_metric("Total_StoreDept_Models", len(results))

    best_pipeline = Pipeline([
        ('model', ProphetWrapper(
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=False,
            changepoint_prior_scale=0.5
        ))
    ])

    mlflow.sklearn.log_model(best_pipeline, artifact_path="Prophet_Pipeline")

    results_df = pd.DataFrame(results)
    results_df.to_csv('/content/drive/MyDrive/prophet_results.csv', index=False)

    print(results_df.head())

print("Done. Model logged and saved.")
mlflow.end_run()


--- Starting Prophet for 3331 Store-Dept combos ---


DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/bklqanfn.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/u1h7_re_.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=6858', 'data', 'file=/tmp/tmp8yx8fhyr/bklqanfn.json', 'init=/tmp/tmp8yx8fhyr/u1h7_re_.json', 'output', 'file=/tmp/tmp8yx8fhyr/prophet_modelwyscsk5t/prophet_model-20250707141133.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:11:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:11:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 1, Dept: 1 (1/3331) ---
   WMAE: 4628.94 | RMSE: 7541.63


DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/vn1ouuco.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/03zpk2hm.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=16022', 'data', 'file=/tmp/tmp8yx8fhyr/vn1ouuco.json', 'init=/tmp/tmp8yx8fhyr/03zpk2hm.json', 'output', 'file=/tmp/tmp8yx8fhyr/prophet_modelgm0at2yn/prophet_model-20250707141133.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:11:33 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:11:33 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 1, Dept: 2 (2/3331) ---
   WMAE: 3301.70 | RMSE: 4227.02


...

DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/rudhpciw.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/f8xbb58p.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=85864', 'data', 'file=/tmp/tmp8yx8fhyr/rudhpciw.json', 'init=/tmp/tmp8yx8fhyr/f8xbb58p.json', 'output', 'file=/tmp/tmp8yx8fhyr/prophet_modeld61mzbb0/prophet_model-20250707142551.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:25:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:25:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 45, Dept: 94 (3327/3331) ---
   Skipped: Not enough data (91 train, 26 val)

--- Processing Store: 45, Dept: 95 (3328/3331) ---
   WMAE: 1595.13 | RMSE: 2492.39


DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/bljfuzu5.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8yx8fhyr/nbiufbig.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=20116', 'data', 'file=/tmp/tmp8yx8fhyr/bljfuzu5.json', 'init=/tmp/tmp8yx8fhyr/nbiufbig.json', 'output', 'file=/tmp/tmp8yx8fhyr/prophet_modelmnijgguv/prophet_model-20250707142551.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
14:25:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
14:25:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing



--- Processing Store: 45, Dept: 96 (3329/3331) ---
   Skipped: Not enough data (0 train, 1 val)

--- Processing Store: 45, Dept: 97 (3330/3331) ---
   WMAE: 892.33 | RMSE: 1054.87



--- Processing Store: 45, Dept: 98 (3331/3331) ---
   Skipped: Not enough data (92 train, 26 val)

 Overall WMAE: 2222.42
   Store  Dept         RMSE         WMAE
0      1     1  7541.630848  4628.941610
1      1     2  4227.016284  3301.700632
2      1     3  1395.065175  1195.011634
3      1     4  2709.434896  2013.208335
4      1     5  4652.433928  3265.800798
🏃 View run Prophet_Improved_Seasonality_Run at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/2/runs/745af4f6f53341faab924253c9ca2a13
🧪 View experiment at: https://dagshub.com/eghib22/Store-Sales-Forecasting.mlflow/#/experiments/2
Done. Model logged and saved.
